In [21]:
from googleapiclient.discovery import build
import pandas as pd
from IPython.display import JSON

In [22]:
api_key = input("Please enter your YouTube Data API key: ")
channel_id='UCtAXJ4DNpshfVjLIMq9pIRw'
youtube= build('youtube','v3',developerKey=api_key)

KeyboardInterrupt: Interrupted by user

In [ ]:
def get_channel(youtube, channel_id):
    all_data = []
    request = youtube.channels().list(
        part='snippet,contentDetails,statistics',
        id=channel_id
    )
    response = request.execute() 
    
    # Check if 'items' key exists in the response
    if 'items' in response:
        for i in range(len(response['items'])):
            data = dict(
                channelName=response['items'][i]['snippet']['title'],
                subscribers=response['items'][i]['statistics']['subscriberCount'],
                views=response['items'][i]['statistics']['viewCount'],
                totalVideos=response['items'][i]['statistics']['videoCount'],
                playlistId=response['items'][i]['contentDetails']['relatedPlaylists']['uploads']
            )
            all_data.append(data)
    else:
        print("No 'items' found in response.")
    
    return pd.DataFrame(all_data)



In [ ]:
channel_stats=get_channel(youtube,channel_id)
channel_stats

,channelName,subscribers,views,totalVideos,playlistId
0,Rohit & Kanupriya,2280000,2166832637,326,UUtAXJ4DNpshfVjLIMq9pIRw


In [ ]:
playlistid=channel_stats['playlistId']
playlistid

0    UUtAXJ4DNpshfVjLIMq9pIRw
Name: playlistId, dtype: object

In [ ]:
def get_video_ids(youtube, playlist_id,video_ids):

    request = youtube.playlistItems().list(
                part='contentDetails',
                playlistId = playlist_id,
                maxResults = 50)
    response = request.execute()
    
    
    for i in range(len(response['items'])):
        video_ids.append(response['items'][i]['contentDetails']['videoId'])
        
    next_page_token = response.get('nextPageToken')
    more_pages = True
    
    while more_pages:
        if next_page_token is None:
            more_pages = False
        else:
            request = youtube.playlistItems().list(
                        part='contentDetails',
                        playlistId = playlist_id,
                        maxResults = 50,
                        pageToken = next_page_token)
            response = request.execute()
    
            for i in range(len(response['items'])):
                video_ids.append(response['items'][i]['contentDetails']['videoId'])
            
            next_page_token = response.get('nextPageToken')
        
    return video_ids

In [ ]:
video_ids = []
for i in range(len(playlistid)):
    playlist_id=playlistid[i]
    Video_ids=get_video_ids(youtube, playlist_id,video_ids)
    
len(Video_ids)

326

In [ ]:
def get_comments_in_videos(youtube, video_ids):
    all_comments = []
    
    for video_id in video_ids:
        try:   
            request = youtube.commentThreads().list(
                part="snippet,replies",
                videoId=video_id
            )
            response = request.execute()
        
            comments_in_video = []
            for comment in response['items']:
                comment_text = comment['snippet']['topLevelComment']['snippet']['textOriginal']
                commenter_name = comment['snippet']['topLevelComment']['snippet']['authorDisplayName']
                comments_in_video.append({'comment': comment_text, 'author': commenter_name})
            
            comments_in_video_info = {'video_id': video_id, 'comments': comments_in_video}
            all_comments.append({
                    'VideoID': video_id,
                    'Username': commenter_name,
                    'Comment': comment_text
                })
            
        except Exception as e: 
            # When error occurs - most likely because comments are disabled on a video
            print(f'Could not get comments for video {video_id}: {e}')
        
    return pd.DataFrame(all_comments)  



In [ ]:
comments = get_comments_in_videos(youtube, video_ids)
comments

,VideoID,Username,Comment
0,yXoqz_GOJxw,@DevaDarshini-eq9nm,😂😂😂
1,QGP02UnMhhE,@Rapid_Wolf_Memes,Bro my parents in front of me and I am watchin...
2,ScnDAe0IsYc,@summer-cq9wx,I have been planning to go to Kodai with frien...
3,UdrlseaiS2E,@vishakr7194,Good one 😂😂😂
4,2WCcSLE7aDA,@nks8630,Wife ke maare do bechare😂😂😂
...,...,...,...
321,TRM_e1O7TVU,@ornatrabelsi9174,So young❤
322,U_xZFqXOyFY,@Uppolice.age.relaxation,The smile at the end 😂
323,k7dmrSbCpy4,@bnm3693,That's sooo me since the beginning of this month
324,cffqyq-U2Og,@Madhuwellness,My 10 year old 😂


In [ ]:
comments.to_csv('Data\\commentsData.csv', index=False)